In [1]:
from pyspark.sql.types import StringType, LongType, TimestampType
from pyspark.sql.window import Window
from pyspark.sql import functions as F

StatementMeta(, d53017c2-cc08-49e4-9366-cedc57f574b2, 3, Finished, Available, Finished)

In [2]:
tabla_categoria = {
    "origen": "project_categoriaActivo",           
    "dimension": "dimensionCategoriaSilver",        
    "campo_clave": "codigocategoria",               
    "columnas_base": ["codigocategoria", "nomcategoria"], 
    "nombre_sk": "skCategoriaActivo"               
}

StatementMeta(, d53017c2-cc08-49e4-9366-cedc57f574b2, 4, Finished, Available, Finished)

In [3]:
invalid_chars = [' ', ',', ';', '{', '}', '(', ')', '\n', '\t', '=']
def limpiar_nombre_columna(nombre):
    for ch in invalid_chars:
        nombre = nombre.replace(ch, '_')
    return nombre

StatementMeta(, d53017c2-cc08-49e4-9366-cedc57f574b2, 5, Finished, Available, Finished)

In [4]:
df_base = spark.sql(f"SELECT * FROM lakehouseSilver.{tabla_categoria['origen']}")
df_base = df_base.dropDuplicates([tabla_categoria["campo_clave"]])

StatementMeta(, d53017c2-cc08-49e4-9366-cedc57f574b2, 6, Finished, Available, Finished)

In [5]:
try:
    df_actual = spark.sql(f"SELECT * FROM lakehouseSilver.{tabla_categoria['dimension']}")
    existe = True
except:
    existe = False
    schema = df_base.schema.add(tabla_categoria["nombre_sk"], LongType()) \
        .add("FechaCreacion", TimestampType()) \
        .add("EsActual", StringType())
    df_actual = spark.createDataFrame([], schema)

StatementMeta(, d53017c2-cc08-49e4-9366-cedc57f574b2, 7, Finished, Available, Finished)

In [6]:
df_nuevos = df_base.join(df_actual.select(tabla_categoria["campo_clave"]), on=tabla_categoria["campo_clave"], how="left_anti")

StatementMeta(, d53017c2-cc08-49e4-9366-cedc57f574b2, 8, Finished, Available, Finished)

In [7]:
if existe and tabla_categoria["nombre_sk"] in df_actual.columns and df_actual.count() > 0:
    max_sk = df_actual.agg(F.max(tabla_categoria["nombre_sk"])).collect()[0][0] or 0
else:
    max_sk = 0

StatementMeta(, d53017c2-cc08-49e4-9366-cedc57f574b2, 9, Finished, Available, Finished)

In [8]:
window = Window.orderBy(tabla_categoria["campo_clave"])
df_nuevos_sk = df_nuevos.withColumn(tabla_categoria["nombre_sk"], F.row_number().over(window) + max_sk)

StatementMeta(, d53017c2-cc08-49e4-9366-cedc57f574b2, 10, Finished, Available, Finished)

In [9]:
now = F.current_timestamp()
df_nuevos_sk = df_nuevos_sk.withColumn("FechaCreacion", now).withColumn("EsActual", F.lit("Sí"))

StatementMeta(, d53017c2-cc08-49e4-9366-cedc57f574b2, 11, Finished, Available, Finished)

In [10]:
for colname in ["FechaCreacion", "EsActual"]:
    if colname not in df_actual.columns:
        df_actual = df_actual.withColumn(colname, F.lit(None).cast(TimestampType() if colname == "FechaCreacion" else StringType()))


StatementMeta(, d53017c2-cc08-49e4-9366-cedc57f574b2, 12, Finished, Available, Finished)

In [11]:
df_final = df_actual.unionByName(df_nuevos_sk)

StatementMeta(, d53017c2-cc08-49e4-9366-cedc57f574b2, 13, Finished, Available, Finished)

In [12]:
df_final = df_final.select([F.col(c).alias(limpiar_nombre_columna(c)) for c in df_final.columns])

StatementMeta(, d53017c2-cc08-49e4-9366-cedc57f574b2, 14, Finished, Available, Finished)

In [13]:
if df_final.groupBy(tabla_categoria["nombre_sk"]).count().filter("count > 1").count() > 0:
    raise Exception(f"❌ Duplicados en {tabla_categoria['nombre_sk']}")

StatementMeta(, d53017c2-cc08-49e4-9366-cedc57f574b2, 15, Finished, Available, Finished)

In [14]:
df_final.write.mode("overwrite").format("delta").saveAsTable(f"lakehouseSilver.{tabla_categoria['dimension']}")

StatementMeta(, d53017c2-cc08-49e4-9366-cedc57f574b2, 16, Finished, Available, Finished)

In [15]:
print(f"✅ {tabla_categoria['dimension']} actualizada correctamente.")

StatementMeta(, d53017c2-cc08-49e4-9366-cedc57f574b2, 17, Finished, Available, Finished)

✅ dimensionCategoriaSilver actualizada correctamente.
